In [1]:
folder_path = '/users/irisz/downloads/2023_elic/ivn_data/ivn_data_0305'
save_path = '/users/irisz/downloads/2023_elic/ivn_data/ivn_finetune_chunk'



In [2]:
# get file name list - wav and textgrid

from os import listdir
from os.path import isfile, join
files = [f for f in listdir(folder_path) if isfile(join(folder_path, f))]
files

['ivn009-2023-02-15-Pula_03.TextGrid',
 'ivn009-2023-02-15-Pula_02.TextGrid',
 'ivn010-2023-02-27-Pula_07.TextGrid',
 'ivn010-2023-02-27-Pula_06.TextGrid',
 'ivn009-2023-02-15-Pula_09.TextGrid',
 'ivn009-2023-02-15-Pula_08.TextGrid',
 '.DS_Store',
 'ivn011-2023-02-25-Rovinj_01.TextGrid',
 'ivn008-2023-02-14-Rovinj_05.TextGrid',
 'ivn008-2023-02-14-Rovinj_04.TextGrid',
 'ivn007-2023-02-09-Rovinj_04.TextGrid',
 'ivn007-2023-02-09-Rovinj_05.TextGrid',
 'ivn006-2023-02-07-Pula_01.TextGrid',
 'ivn009-2023-02-15-Pula_04.wav',
 'ivn009-2023-02-15-Pula_10.wav',
 'ivn011-2023-02-25-Rovinj_06.wav',
 'ivn008-2023-02-14-Rovinj_01.wav',
 'ivn011-2023-02-25-Rovinj_06.TextGrid',
 'ivn011-2023-02-25-Rovinj_07.TextGrid',
 'ivn008-2023-02-14-Rovinj_02.TextGrid',
 'ivn008-2023-02-14-Rovinj_03.TextGrid',
 'ivn009-2023-02-15-Pula_11.wav',
 'ivn011-2023-02-25-Rovinj_07.wav',
 'ivn009-2023-02-15-Pula_05.wav',
 'ivn007-2023-02-09-Rovinj_03.TextGrid',
 'ivn007-2023-02-09-Rovinj_02.TextGrid',
 'ivn011-2023-02-2

In [3]:
# subset of the files that 
file_names = []
for file in files:
    if file.startswith('i'):
        file_names.append(file.split('.')[0])
file_names = list(set(file_names))
file_names

['ivn011-2023-02-25-Rovinj_03',
 'ivn007-2023-02-09-Rovinj_01',
 'ivn008-2023-02-14-Rovinj_07',
 'ivn008-2023-02-14-Rovinj_04',
 'ivn010-2023-02-27-Pula_01',
 'ivn007-2023-02-09-Rovinj_06',
 'ivn009-2023-02-15-Pula_11',
 'ivn009-2023-02-15-Pula_08',
 'ivn006-2023-02-07-Pula_04',
 'ivn006-2023-02-07-Pula_01',
 'ivn008-2023-02-14-Rovinj_01',
 'ivn009-2023-02-15-Pula_02',
 'ivn008-2023-02-14-Rovinj_06',
 'ivn006-2023-02-07-Pula_03',
 'ivn010-2023-02-27-Pula_07',
 'ivn009-2023-02-15-Pula_07',
 'ivn008-2023-02-14-Rovinj_09',
 'ivn011-2023-02-25-Rovinj_05',
 'ivn009-2023-02-15-Pula_04',
 'ivn011-2023-02-25-Rovinj_02',
 'ivn010-2023-02-27-Pula_03',
 'ivn007-2023-02-09-Rovinj_02',
 'ivn007-2023-02-09-Rovinj_03',
 'ivn010-2023-02-27-Pula_06',
 'ivn009-2023-02-15-Pula_03',
 'ivn011-2023-02-25-Rovinj_06',
 'ivn008-2023-02-14-Rovinj_03',
 'ivn009-2023-02-15-Pula_09',
 'ivn011-2023-02-25-Rovinj_01',
 'ivn008-2023-02-14-Rovinj_05',
 'ivn010-2023-02-27-Pula_02',
 'ivn009-2023-02-15-Pula_06',
 'ivn011

In [4]:

# get original time info from textgrid

import re


def get_interval(time_list):
    interval_all = []
    interval_curr = 1
    for i in range(len(time_list)):
        if i == 0:
            interval_all.append(1)
        else:
            if time_list[i] > time_list[i - 1]:
                interval_all.append(interval_curr)
                continue
            else:
                interval_curr = interval_curr + 1
                interval_all.append(interval_curr)
    return interval_all


def get_info_from_textgrid(textgrid_file, file_name):
    #with open(textgrid_file, 'r') as file:
    try:    # file encoding may be different, try here 
        with open(textgrid_file, encoding = "utf-8") as file:
            lines = [line.rstrip() for line in file]
    except:
        with open(textgrid_file, encoding = "utf-16") as file:
            lines = [line.rstrip() for line in file]
   
    text_list = []
    xmin_list = []
    xmax_list = []
    interval_list = []
    for line in lines[14:]:  #informations needed begin on the 9th lines
        if 'text =' in line:
            line = line.split('"')[1]
            if line != "":
                text_list.append(line)
                #print (len(text_list), len(xmin_list), len(xmax_list), len(interval_list), line, interval_list[-2:])
            else:
                xmin_list = xmin_list[:-1]
                xmax_list = xmax_list[:-1]
                interval_list = interval_list[:-1]
        if 'xmin' in line:
            time_min = line.split('=')[1].strip(' ')
            xmin_list.append(time_min)
        if 'xmax' in line:
            time_max = line.split('=')[1].strip(' ')
            xmax_list.append(time_max)
        if 'intervals [' in line:
            interval = line.split('[')[1].strip(']:')
            interval_list.append(interval)
        if 'intervals: size' in line:
            xmin_list = xmin_list[:-1]
            xmax_list = xmax_list[:-1]
    file_name_list = [file_name] * len(text_list)
    
    xmin_list = [float(item) for item in xmin_list]
    xmax_list = [float(item) for item in xmax_list]
    interval_list = [int(item) for item in interval_list] # annotation chunk
    
    tier_list = get_interval(interval_list)
    return text_list, xmin_list, xmax_list, interval_list, file_name_list, tier_list
        

In [5]:
text_list = []
xmin_list = []
xmax_list = []
interval_list = []
file_name_list = []
tier_list = []

for file_name in file_names:
    print (file_name)
    textgrid_file = folder_path + '/' + file_name + '.TextGrid' # file path for textgrid
    file_info_list = get_info_from_textgrid(textgrid_file, file_name)
    text_list = text_list + file_info_list[0]
    xmin_list = xmin_list + file_info_list[1]
    xmax_list = xmax_list + file_info_list[2]
    interval_list = interval_list + file_info_list[3]
    file_name_list = file_name_list + file_info_list[4]
    tier_list = tier_list + file_info_list[5]

ivn011-2023-02-25-Rovinj_03
ivn007-2023-02-09-Rovinj_01
ivn008-2023-02-14-Rovinj_07
ivn008-2023-02-14-Rovinj_04
ivn010-2023-02-27-Pula_01
ivn007-2023-02-09-Rovinj_06
ivn009-2023-02-15-Pula_11
ivn009-2023-02-15-Pula_08
ivn006-2023-02-07-Pula_04
ivn006-2023-02-07-Pula_01
ivn008-2023-02-14-Rovinj_01
ivn009-2023-02-15-Pula_02
ivn008-2023-02-14-Rovinj_06
ivn006-2023-02-07-Pula_03
ivn010-2023-02-27-Pula_07
ivn009-2023-02-15-Pula_07
ivn008-2023-02-14-Rovinj_09
ivn011-2023-02-25-Rovinj_05
ivn009-2023-02-15-Pula_04
ivn011-2023-02-25-Rovinj_02
ivn010-2023-02-27-Pula_03
ivn007-2023-02-09-Rovinj_02
ivn007-2023-02-09-Rovinj_03
ivn010-2023-02-27-Pula_06
ivn009-2023-02-15-Pula_03
ivn011-2023-02-25-Rovinj_06
ivn008-2023-02-14-Rovinj_03
ivn009-2023-02-15-Pula_09
ivn011-2023-02-25-Rovinj_01
ivn008-2023-02-14-Rovinj_05
ivn010-2023-02-27-Pula_02
ivn009-2023-02-15-Pula_06
ivn011-2023-02-25-Rovinj_04
ivn009-2023-02-15-Pula_01
ivn011-2023-02-25-Rovinj_08
ivn008-2023-02-14-Rovinj_02
ivn011-2023-02-25-Rovinj_0

In [6]:
# save text and time info in dataframe

import pandas as pd

df = pd.DataFrame()
df['file'] = file_name_list
df['text'] = text_list
df['xmin'] = xmin_list
df['xmax'] = xmax_list
df['interval'] = interval_list
df['tier'] = tier_list

In [7]:
df

,file,text,xmin,xmax,interval,tier
0,ivn011-2023-02-25-Rovinj_03,Ah. M-hm.,10.354614,11.565379,2,1
1,ivn011-2023-02-25-Rovinj_03,I ciama pojačanja per vinzer e non per parteci...,62.177870,68.087574,4,1
2,ivn011-2023-02-25-Rovinj_03,"In teoria, mi gavessi 'sto preconcetto che chi...",91.988884,106.124379,6,1
3,ivn011-2023-02-25-Rovinj_03,"Co ti va per boteghe, banche e poste, cussì- N...",174.571169,179.231026,8,1
4,ivn011-2023-02-25-Rovinj_03,"Te vien de saludar, @ incominciar in- in diale...",185.161761,190.070606,10,1
...,...,...,...,...,...,...
3566,ivn007-2023-02-09-Rovinj_05,Grazie ((laugh)).,528.478089,529.378438,105,2
3567,ivn007-2023-02-09-Rovinj_05,"Sì, sì, sì. Me x-",535.568677,537.417671,107,2
3568,ivn007-2023-02-09-Rovinj_05,@ no voleria gufar ((breath)) però penso che- ...,544.607440,572.418984,109,2
3569,ivn007-2023-02-09-Rovinj_05,"N- no, no, non- non me vedo el tipo de lavorar...",574.314667,579.389833,111,2


In [8]:
set(df['file'].tolist()) # all file covered

{'ivn006-2023-02-07-Pula_01',
 'ivn006-2023-02-07-Pula_02',
 'ivn006-2023-02-07-Pula_03',
 'ivn006-2023-02-07-Pula_04',
 'ivn006-2023-02-07-Pula_05',
 'ivn007-2023-02-09-Rovinj_01',
 'ivn007-2023-02-09-Rovinj_02',
 'ivn007-2023-02-09-Rovinj_03',
 'ivn007-2023-02-09-Rovinj_04',
 'ivn007-2023-02-09-Rovinj_05',
 'ivn007-2023-02-09-Rovinj_06',
 'ivn008-2023-02-14-Rovinj_01',
 'ivn008-2023-02-14-Rovinj_02',
 'ivn008-2023-02-14-Rovinj_03',
 'ivn008-2023-02-14-Rovinj_04',
 'ivn008-2023-02-14-Rovinj_05',
 'ivn008-2023-02-14-Rovinj_06',
 'ivn008-2023-02-14-Rovinj_07',
 'ivn008-2023-02-14-Rovinj_08',
 'ivn008-2023-02-14-Rovinj_09',
 'ivn009-2023-02-15-Pula_01',
 'ivn009-2023-02-15-Pula_02',
 'ivn009-2023-02-15-Pula_03',
 'ivn009-2023-02-15-Pula_04',
 'ivn009-2023-02-15-Pula_05',
 'ivn009-2023-02-15-Pula_06',
 'ivn009-2023-02-15-Pula_07',
 'ivn009-2023-02-15-Pula_08',
 'ivn009-2023-02-15-Pula_09',
 'ivn009-2023-02-15-Pula_10',
 'ivn009-2023-02-15-Pula_11',
 'ivn010-2023-02-27-Pula_01',
 'ivn010-2

In [9]:
# classify word or sentence level

def check_word(text):
    if '.' in text or ',' in text or ' ' in text or '?' in text or '!' in text:
        return 1
    else:
        return 0
    

In [10]:
df['sent'] = df['text'].apply(lambda x: check_word(x))
df

,file,text,xmin,xmax,interval,tier,sent
0,ivn011-2023-02-25-Rovinj_03,Ah. M-hm.,10.354614,11.565379,2,1,1
1,ivn011-2023-02-25-Rovinj_03,I ciama pojačanja per vinzer e non per parteci...,62.177870,68.087574,4,1,1
2,ivn011-2023-02-25-Rovinj_03,"In teoria, mi gavessi 'sto preconcetto che chi...",91.988884,106.124379,6,1,1
3,ivn011-2023-02-25-Rovinj_03,"Co ti va per boteghe, banche e poste, cussì- N...",174.571169,179.231026,8,1,1
4,ivn011-2023-02-25-Rovinj_03,"Te vien de saludar, @ incominciar in- in diale...",185.161761,190.070606,10,1,1
...,...,...,...,...,...,...,...
3566,ivn007-2023-02-09-Rovinj_05,Grazie ((laugh)).,528.478089,529.378438,105,2,1
3567,ivn007-2023-02-09-Rovinj_05,"Sì, sì, sì. Me x-",535.568677,537.417671,107,2,1
3568,ivn007-2023-02-09-Rovinj_05,@ no voleria gufar ((breath)) però penso che- ...,544.607440,572.418984,109,2,1
3569,ivn007-2023-02-09-Rovinj_05,"N- no, no, non- non me vedo el tipo de lavorar...",574.314667,579.389833,111,2,1


In [46]:
a = [1,1,0]
a.count(1)

2

In [11]:
# get tier word percentage
tier_type = []

for file_name in file_names:
    #print (file_name)
    df_sub = df[df['file'] == file_name]# choose subset with file name
    tiers = list(set(df_sub['tier'].tolist()))
    for tier in tiers:
        df_sub_curr = df_sub[df_sub['tier'] == tier]
        sent_type_list = df_sub_curr['sent'].tolist()
        tier_sent_percent = sent_type_list.count(1)/len(sent_type_list)
        print (file_name, tier, tier_sent_percent, len(sent_type_list))
        if tier_sent_percent > 0.5: # current layer is a sentence layer
            tier_type = tier_type + [1]*len(df_sub_curr)
        else:
            tier_type = tier_type + [0]*len(df_sub_curr)

ivn011-2023-02-25-Rovinj_03 1 1.0 11
ivn011-2023-02-25-Rovinj_03 2 0.725 40
ivn007-2023-02-09-Rovinj_01 1 0.9523809523809523 42
ivn007-2023-02-09-Rovinj_01 2 1.0 44
ivn008-2023-02-14-Rovinj_07 1 0.8571428571428571 14
ivn008-2023-02-14-Rovinj_07 2 0.5949367088607594 79
ivn008-2023-02-14-Rovinj_04 1 0.9444444444444444 18
ivn008-2023-02-14-Rovinj_04 2 0.9636363636363636 55
ivn010-2023-02-27-Pula_01 1 0.9130434782608695 23
ivn010-2023-02-27-Pula_01 2 0.6530612244897959 49
ivn007-2023-02-09-Rovinj_06 1 1.0 12
ivn007-2023-02-09-Rovinj_06 2 0.9583333333333334 24
ivn009-2023-02-15-Pula_11 1 0.8333333333333334 54
ivn009-2023-02-15-Pula_11 2 0.9347826086956522 46
ivn009-2023-02-15-Pula_08 1 0.972972972972973 37
ivn009-2023-02-15-Pula_08 2 1.0 63
ivn006-2023-02-07-Pula_04 1 0.9791666666666666 48
ivn006-2023-02-07-Pula_04 2 0.9787234042553191 47
ivn006-2023-02-07-Pula_01 1 0.8936170212765957 47
ivn006-2023-02-07-Pula_01 2 1.0 44
ivn008-2023-02-14-Rovinj_01 1 0.8421052631578947 19
ivn008-2023-02-14

In [12]:
len(tier_type)

3571

In [13]:
df['tier_sent'] = tier_type
df # tier sent marked, and checked correct

,file,text,xmin,xmax,interval,tier,sent,tier_sent
0,ivn011-2023-02-25-Rovinj_03,Ah. M-hm.,10.354614,11.565379,2,1,1,1
1,ivn011-2023-02-25-Rovinj_03,I ciama pojačanja per vinzer e non per parteci...,62.177870,68.087574,4,1,1,1
2,ivn011-2023-02-25-Rovinj_03,"In teoria, mi gavessi 'sto preconcetto che chi...",91.988884,106.124379,6,1,1,1
3,ivn011-2023-02-25-Rovinj_03,"Co ti va per boteghe, banche e poste, cussì- N...",174.571169,179.231026,8,1,1,1
4,ivn011-2023-02-25-Rovinj_03,"Te vien de saludar, @ incominciar in- in diale...",185.161761,190.070606,10,1,1,1
...,...,...,...,...,...,...,...,...
3566,ivn007-2023-02-09-Rovinj_05,Grazie ((laugh)).,528.478089,529.378438,105,2,1,1
3567,ivn007-2023-02-09-Rovinj_05,"Sì, sì, sì. Me x-",535.568677,537.417671,107,2,1,1
3568,ivn007-2023-02-09-Rovinj_05,@ no voleria gufar ((breath)) però penso che- ...,544.607440,572.418984,109,2,1,1
3569,ivn007-2023-02-09-Rovinj_05,"N- no, no, non- non me vedo el tipo de lavorar...",574.314667,579.389833,111,2,1,1


In [14]:
df['chunk_len'] = df.apply(lambda x: x.xmax - x.xmin, axis = 1)
df

,file,text,xmin,xmax,interval,tier,sent,tier_sent,chunk_len
0,ivn011-2023-02-25-Rovinj_03,Ah. M-hm.,10.354614,11.565379,2,1,1,1,1.210766
1,ivn011-2023-02-25-Rovinj_03,I ciama pojačanja per vinzer e non per parteci...,62.177870,68.087574,4,1,1,1,5.909704
2,ivn011-2023-02-25-Rovinj_03,"In teoria, mi gavessi 'sto preconcetto che chi...",91.988884,106.124379,6,1,1,1,14.135496
3,ivn011-2023-02-25-Rovinj_03,"Co ti va per boteghe, banche e poste, cussì- N...",174.571169,179.231026,8,1,1,1,4.659856
4,ivn011-2023-02-25-Rovinj_03,"Te vien de saludar, @ incominciar in- in diale...",185.161761,190.070606,10,1,1,1,4.908845
...,...,...,...,...,...,...,...,...,...
3566,ivn007-2023-02-09-Rovinj_05,Grazie ((laugh)).,528.478089,529.378438,105,2,1,1,0.900349
3567,ivn007-2023-02-09-Rovinj_05,"Sì, sì, sì. Me x-",535.568677,537.417671,107,2,1,1,1.848995
3568,ivn007-2023-02-09-Rovinj_05,@ no voleria gufar ((breath)) però penso che- ...,544.607440,572.418984,109,2,1,1,27.811544
3569,ivn007-2023-02-09-Rovinj_05,"N- no, no, non- non me vedo el tipo de lavorar...",574.314667,579.389833,111,2,1,1,5.075166


In [15]:
df.to_csv(save_path + '/ivn_text_info.csv')

In [16]:
# get sentence segmentation 

df_sent = df[df['tier_sent'] == 1] # ignore the word tiers
df_sent.reset_index(inplace = True)
df_sent

,index,file,text,xmin,xmax,interval,tier,sent,tier_sent,chunk_len
0,0,ivn011-2023-02-25-Rovinj_03,Ah. M-hm.,10.354614,11.565379,2,1,1,1,1.210766
1,1,ivn011-2023-02-25-Rovinj_03,I ciama pojačanja per vinzer e non per parteci...,62.177870,68.087574,4,1,1,1,5.909704
2,2,ivn011-2023-02-25-Rovinj_03,"In teoria, mi gavessi 'sto preconcetto che chi...",91.988884,106.124379,6,1,1,1,14.135496
3,3,ivn011-2023-02-25-Rovinj_03,"Co ti va per boteghe, banche e poste, cussì- N...",174.571169,179.231026,8,1,1,1,4.659856
4,4,ivn011-2023-02-25-Rovinj_03,"Te vien de saludar, @ incominciar in- in diale...",185.161761,190.070606,10,1,1,1,4.908845
...,...,...,...,...,...,...,...,...,...,...
3566,3566,ivn007-2023-02-09-Rovinj_05,Grazie ((laugh)).,528.478089,529.378438,105,2,1,1,0.900349
3567,3567,ivn007-2023-02-09-Rovinj_05,"Sì, sì, sì. Me x-",535.568677,537.417671,107,2,1,1,1.848995
3568,3568,ivn007-2023-02-09-Rovinj_05,@ no voleria gufar ((breath)) però penso che- ...,544.607440,572.418984,109,2,1,1,27.811544
3569,3569,ivn007-2023-02-09-Rovinj_05,"N- no, no, non- non me vedo el tipo de lavorar...",574.314667,579.389833,111,2,1,1,5.075166


In [49]:
non_sent_list = ['((breath))',
                '((laughs))',
                '((breaths)',
                '@',
                '((coughs))',
                '((click))',
                '(())',
                '#',
                'x',
                'Mhm.',
                'Mhm, mhm.',
                '((hhh))',
                'Mhm',
                 '((noise))',                            
'((overlapping, unintelligible))',      
'((breath, smack))',  
'((breath, coughs))',                   
'((breath)) ((click))',                 
'(x)',            
'((machine sounds))',                   
'((breath, noise))',                                                
'((breath, click))' 
                ]

In [16]:
df_sent['text'].value_counts()[:40]

((breath))                         924
Mhm.                               137
((laughs))                          59
((breaths))                         57
Da.                                 48
@                                   47
Dobro.                              15
((coughs))                          14
((click))                           12
#                                   10
(())                                10
Ja.                                  9
x                                    9
Mhm, mhm.                            8
I tako.                              7
((hhh))                              7
Mhm                                  6
((noise))                            6
Tako da-                             6
Tako.                                6
Ne.                                  5
Aha.                                 5
((overlapping, unintelligible))      5
((breath, smack))                    5
((breath)) ((click))                 4
((breath, coughs))       

In [21]:
import numpy as np
np.max(df_sent_2['chunk_len'])

44.51318514958736

In [22]:
len(xmin_list)

9812

In [19]:
df_sent['file_tier'] = df_sent.apply(lambda x: x.file + '_' + str(x.tier), axis = 1)
df_sent

,index,file,text,xmin,xmax,interval,tier,sent,tier_sent,chunk_len,file_tier
0,0,ivn011-2023-02-25-Rovinj_03,Ah. M-hm.,10.354614,11.565379,2,1,1,1,1.210766,ivn011-2023-02-25-Rovinj_03_1
1,1,ivn011-2023-02-25-Rovinj_03,I ciama pojačanja per vinzer e non per parteci...,62.177870,68.087574,4,1,1,1,5.909704,ivn011-2023-02-25-Rovinj_03_1
2,2,ivn011-2023-02-25-Rovinj_03,"In teoria, mi gavessi 'sto preconcetto che chi...",91.988884,106.124379,6,1,1,1,14.135496,ivn011-2023-02-25-Rovinj_03_1
3,3,ivn011-2023-02-25-Rovinj_03,"Co ti va per boteghe, banche e poste, cussì- N...",174.571169,179.231026,8,1,1,1,4.659856,ivn011-2023-02-25-Rovinj_03_1
4,4,ivn011-2023-02-25-Rovinj_03,"Te vien de saludar, @ incominciar in- in diale...",185.161761,190.070606,10,1,1,1,4.908845,ivn011-2023-02-25-Rovinj_03_1
...,...,...,...,...,...,...,...,...,...,...,...
3566,3566,ivn007-2023-02-09-Rovinj_05,Grazie ((laugh)).,528.478089,529.378438,105,2,1,1,0.900349,ivn007-2023-02-09-Rovinj_05_2
3567,3567,ivn007-2023-02-09-Rovinj_05,"Sì, sì, sì. Me x-",535.568677,537.417671,107,2,1,1,1.848995,ivn007-2023-02-09-Rovinj_05_2
3568,3568,ivn007-2023-02-09-Rovinj_05,@ no voleria gufar ((breath)) però penso che- ...,544.607440,572.418984,109,2,1,1,27.811544,ivn007-2023-02-09-Rovinj_05_2
3569,3569,ivn007-2023-02-09-Rovinj_05,"N- no, no, non- non me vedo el tipo de lavorar...",574.314667,579.389833,111,2,1,1,5.075166,ivn007-2023-02-09-Rovinj_05_2


In [50]:
df_sent_2 = df_sent[df_sent['text'].isin(non_sent_list) == False]
#df_sent_2 = df_sent

df_sent_2.reset_index(inplace = True, drop = True)
df_sent_2

,index,file,text,xmin,xmax,interval,tier,sent,tier_sent,chunk_len,file_tier,text_clean
0,0,ckm002-2022-01-04-Crikvenica_05,Ste voleli školu?,31.204190,32.248673,2,1,1,1,1.044483,ckm002-2022-01-04-Crikvenica_05_1,Ste voleli školu?
1,1,ckm002-2022-01-04-Crikvenica_05,Dal' van je baš neka njegova anegdota ili neč ...,65.324119,69.378564,4,1,1,1,4.054446,ckm002-2022-01-04-Crikvenica_05_1,Dal' van je baš neka njegova anegdota ili neč ...
2,2,ckm002-2022-01-04-Crikvenica_05,"Da, neč da vas je baš onako- prati kroz život ...",70.839682,75.777303,6,1,1,1,4.937620,ckm002-2022-01-04-Crikvenica_05_1,"Da, neč da vas je baš onako- prati kroz život ..."
3,4,ckm002-2022-01-04-Crikvenica_05,"A kad ste malo divojke porasle, kako je to neg...",172.354525,176.279788,10,1,1,1,3.925263,ckm002-2022-01-04-Crikvenica_05_1,"A kad ste malo divojke porasle, kako je to neg..."
4,5,ckm002-2022-01-04-Crikvenica_05,"To su bili pljesnjaci, kamo se hodilo?",176.279788,178.572342,11,1,1,1,2.292554,ckm002-2022-01-04-Crikvenica_05_1,"To su bili pljesnjaci, kamo se hodilo?"
...,...,...,...,...,...,...,...,...,...,...,...,...
4321,9807,ckm004-2022-07-07-Cres_04,"Ja nisan smela, ono kako bi se šlo vanka i se,...",561.102396,566.427729,271,2,1,1,5.325334,ckm004-2022-07-07-Cres_04_2,"Ja nisan smela, ono kako bi se šlo vanka i se,..."
4322,9808,ckm004-2022-07-07-Cres_04,"Evo, tako da.",566.427729,567.333291,272,2,1,1,0.905561,ckm004-2022-07-07-Cres_04_2,"Evo, tako da."
4323,9809,ckm004-2022-07-07-Cres_04,"Sad se to, ja misin, više ne drži. I va Došašć...",567.744910,572.287601,274,2,1,1,4.542691,ckm004-2022-07-07-Cres_04_2,"Sad se to, ja misin, više ne drži. I va Došašć..."
4324,9810,ckm004-2022-07-07-Cres_04,I ženit.,572.834673,573.488118,276,2,1,1,0.653445,ckm004-2022-07-07-Cres_04_2,I ženit.


# clean the text (())

In [51]:
symbol_list = ['((breath))',
                '((laughs))',
                '((breaths)',
               '((breaths))',
                '((coughs))',
                '((click))',
                '(())',
                '#',
                 '((noise))',                            
'((overlapping, unintelligible))',      
'((breath, smack))',  
'((breath, coughs))',                   
'((breath)) ((click))',                 
'(x)',            
'((machine sounds))',                   
'((breath, noise))',                                               
'((breath, click))' 
                ]

In [28]:
def string_clean(text):
    for symbol in symbol_list:
        text = text.replace(symbol, '@')
    
    return text

text = "xdjii ((breath)) jisjdioj"
string_clean(text)

'xdjii @ jisjdioj'

In [52]:
df_sent_2['text_clean'] = df_sent_2['text'].apply(lambda x: string_clean(x))

<ipython-input-52-1a273dae82a4>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sent_2['text_clean'] = df_sent_2['text'].apply(lambda x: string_clean(x))


In [53]:
df_sent_2

,index,file,text,xmin,xmax,interval,tier,sent,tier_sent,chunk_len,file_tier,text_clean
0,0,ckm002-2022-01-04-Crikvenica_05,Ste voleli školu?,31.204190,32.248673,2,1,1,1,1.044483,ckm002-2022-01-04-Crikvenica_05_1,Ste voleli školu?
1,1,ckm002-2022-01-04-Crikvenica_05,Dal' van je baš neka njegova anegdota ili neč ...,65.324119,69.378564,4,1,1,1,4.054446,ckm002-2022-01-04-Crikvenica_05_1,Dal' van je baš neka njegova anegdota ili neč ...
2,2,ckm002-2022-01-04-Crikvenica_05,"Da, neč da vas je baš onako- prati kroz život ...",70.839682,75.777303,6,1,1,1,4.937620,ckm002-2022-01-04-Crikvenica_05_1,"Da, neč da vas je baš onako- prati kroz život ..."
3,4,ckm002-2022-01-04-Crikvenica_05,"A kad ste malo divojke porasle, kako je to neg...",172.354525,176.279788,10,1,1,1,3.925263,ckm002-2022-01-04-Crikvenica_05_1,"A kad ste malo divojke porasle, kako je to neg..."
4,5,ckm002-2022-01-04-Crikvenica_05,"To su bili pljesnjaci, kamo se hodilo?",176.279788,178.572342,11,1,1,1,2.292554,ckm002-2022-01-04-Crikvenica_05_1,"To su bili pljesnjaci, kamo se hodilo?"
...,...,...,...,...,...,...,...,...,...,...,...,...
4321,9807,ckm004-2022-07-07-Cres_04,"Ja nisan smela, ono kako bi se šlo vanka i se,...",561.102396,566.427729,271,2,1,1,5.325334,ckm004-2022-07-07-Cres_04_2,"Ja nisan smela, ono kako bi se šlo vanka i se,..."
4322,9808,ckm004-2022-07-07-Cres_04,"Evo, tako da.",566.427729,567.333291,272,2,1,1,0.905561,ckm004-2022-07-07-Cres_04_2,"Evo, tako da."
4323,9809,ckm004-2022-07-07-Cres_04,"Sad se to, ja misin, više ne drži. I va Došašć...",567.744910,572.287601,274,2,1,1,4.542691,ckm004-2022-07-07-Cres_04_2,"Sad se to, ja misin, više ne drži. I va Došašć..."
4324,9810,ckm004-2022-07-07-Cres_04,I ženit.,572.834673,573.488118,276,2,1,1,0.653445,ckm004-2022-07-07-Cres_04_2,I ženit.


In [21]:
l1 = df_sent['file_tier'].tolist()

file_tier_list = sorted(set(l1), key=l1.index)

In [22]:
file_tier_list

['ivn011-2023-02-25-Rovinj_03_1',
 'ivn011-2023-02-25-Rovinj_03_2',
 'ivn007-2023-02-09-Rovinj_01_1',
 'ivn007-2023-02-09-Rovinj_01_2',
 'ivn008-2023-02-14-Rovinj_07_1',
 'ivn008-2023-02-14-Rovinj_07_2',
 'ivn008-2023-02-14-Rovinj_04_1',
 'ivn008-2023-02-14-Rovinj_04_2',
 'ivn010-2023-02-27-Pula_01_1',
 'ivn010-2023-02-27-Pula_01_2',
 'ivn007-2023-02-09-Rovinj_06_1',
 'ivn007-2023-02-09-Rovinj_06_2',
 'ivn009-2023-02-15-Pula_11_1',
 'ivn009-2023-02-15-Pula_11_2',
 'ivn009-2023-02-15-Pula_08_1',
 'ivn009-2023-02-15-Pula_08_2',
 'ivn006-2023-02-07-Pula_04_1',
 'ivn006-2023-02-07-Pula_04_2',
 'ivn006-2023-02-07-Pula_01_1',
 'ivn006-2023-02-07-Pula_01_2',
 'ivn008-2023-02-14-Rovinj_01_1',
 'ivn008-2023-02-14-Rovinj_01_2',
 'ivn009-2023-02-15-Pula_02_1',
 'ivn009-2023-02-15-Pula_02_2',
 'ivn008-2023-02-14-Rovinj_06_1',
 'ivn008-2023-02-14-Rovinj_06_2',
 'ivn006-2023-02-07-Pula_03_1',
 'ivn006-2023-02-07-Pula_03_2',
 'ivn010-2023-02-27-Pula_07_1',
 'ivn010-2023-02-27-Pula_07_2',
 'ivn009-202

# read the textgrid file and get reasonable chunk time points


In [17]:
def locate_end(xmin_list, xmax_list, i):
    start_curr = xmin_list[i]
    if xmax_list[-1] - start_curr <= 20: # if the end of max < 15, return the last xmax
        return xmax_list[-1], len(xmin_list) + 1
    if xmin_list[i + 1] - start_curr > 30: # if next start leaves too long of blank
        return start_curr + 20, i + 1
    while i < len(xmin_list):
        end_curr = xmax_list[i]
        #print ('----', end_curr, i)
        if end_curr - start_curr >= 20:
            if end_curr - start_curr <= 25:
                return end_curr, i + 1
            else:
                return start_curr + (25 - (xmax_list[i-1] - start_curr)), i #???
        else:
            i = i + 1
            continue
    

In [24]:
def locate_end_small(xmin_list, xmax_list, i):
    start_curr = xmin_list[i]
    end_curr = xmax_list[i]

    if xmax_list[-1] - start_curr <= 20: # if the end of max < 20, return the last xmax
        return xmax_list[-1], len(xmin_list) + 1

    if end_curr - start_curr >= 20:
        return end_curr, i + 1
    
    while i < len(xmin_list):
        end_curr = xmax_list[i]
        #print ('----', start_curr, end_curr, i)
        if end_curr - start_curr >= 20:
            
            if xmin_list[i] - start_curr >= 20: # current start - last end > 20 blank, sep the blank and extend the last one
                return start_curr + 18,i
            else:
                return end_curr, i + 1
      
        else:
            i = i + 1
            continue

In [18]:
df_sent

,index,file,text,xmin,xmax,interval,tier,sent,tier_sent,chunk_len
0,0,ivn011-2023-02-25-Rovinj_03,Ah. M-hm.,10.354614,11.565379,2,1,1,1,1.210766
1,1,ivn011-2023-02-25-Rovinj_03,I ciama pojačanja per vinzer e non per parteci...,62.177870,68.087574,4,1,1,1,5.909704
2,2,ivn011-2023-02-25-Rovinj_03,"In teoria, mi gavessi 'sto preconcetto che chi...",91.988884,106.124379,6,1,1,1,14.135496
3,3,ivn011-2023-02-25-Rovinj_03,"Co ti va per boteghe, banche e poste, cussì- N...",174.571169,179.231026,8,1,1,1,4.659856
4,4,ivn011-2023-02-25-Rovinj_03,"Te vien de saludar, @ incominciar in- in diale...",185.161761,190.070606,10,1,1,1,4.908845
...,...,...,...,...,...,...,...,...,...,...
3566,3566,ivn007-2023-02-09-Rovinj_05,Grazie ((laugh)).,528.478089,529.378438,105,2,1,1,0.900349
3567,3567,ivn007-2023-02-09-Rovinj_05,"Sì, sì, sì. Me x-",535.568677,537.417671,107,2,1,1,1.848995
3568,3568,ivn007-2023-02-09-Rovinj_05,@ no voleria gufar ((breath)) però penso che- ...,544.607440,572.418984,109,2,1,1,27.811544
3569,3569,ivn007-2023-02-09-Rovinj_05,"N- no, no, non- non me vedo el tipo de lavorar...",574.314667,579.389833,111,2,1,1,5.075166


In [25]:
# chunk size 15 - 20 seconds

start = []
end = []
start_id = []
end_id = []
file_info = []
chunk_name = []

for file_name in file_tier_list:
#for file_name in [file_names[0]]:
    print (file_name)

    df_curr = df_sent[df_sent['file_tier'] == file_name] # change df info
    xmin_list = df_curr['xmin'].tolist()
    xmax_list = df_curr['xmax'].tolist()
    
    i = 0
    j = 0 # counting the file chunk number
    while i < len(xmin_list):
        start_curr = xmin_list[i]
        i_curr = i
        # locate the one that > 15 and  < 25 in endlist
        end_curr = locate_end_small(xmin_list, xmax_list, i)[0] # change locate end function
        i = locate_end_small(xmin_list, xmax_list, i)[1]
        print (start_curr, end_curr, i_curr, i)
        
        j = j + 1 # chunk number update
        
        start.append(start_curr)
        end.append(end_curr)
        start_id.append(i_curr)
        end_id.append(i)
        file_info.append(file_name)
        chunk_name.append(file_name + '_' + str(j))

ivn011-2023-02-25-Rovinj_03_1
10.354613646920287 28.354613646920285 0 1
62.17786962159034 80.17786962159033 1 2
91.98888380422831 109.98888380422831 2 3
174.57116932201305 192.57116932201305 3 5
381.90077181263035 399.90077181263035 5 6
420.46670564705977 454.9115026404955 6 9
455.9932573710001 473.9932573710001 9 10
584.6409312320951 586.2055253896219 10 12
ivn011-2023-02-25-Rovinj_03_2
0.0 62.38367904936706 0 2
63.485879920908744 91.48979594186976 2 3
98.91981214997325 143.92001903152087 3 5
143.92001903152087 174.3679325120835 5 8
175.90448563745252 224.79428850136821 8 11
224.79428850136821 248.64391413742345 11 13
248.64391413742345 277.16568888963633 13 14
277.16568888963633 321.34328042503773 14 18
321.34328042503773 345.43959213869846 18 20
345.43959213869846 381.90077181263035 20 22
382.07411233483595 420.54115646258504 22 25
421.85112681033905 443.20898004838716 25 28
455.2613786677159 480.93533799143114 28 30
480.93533799143114 504.96378651462226 30 32
504.96378651462226 544

253.03104910853588 276.41329216407047 26 29
278.48549756267664 313.3582918632075 29 31
314.39134172589445 344.10424561703763 31 33
345.57807127783747 367.9994922720042 33 34
368.8214078561858 392.6692298455729 34 35
394.5577835534368 414.9478789968775 35 42
415.2872019444754 453.40244581031527 42 47
453.9161909912885 471.9161909912885 47 48
474.8769653157913 475.4877466214675 48 50
ivn006-2023-02-07-Pula_03_1
0.0 18.0 0 2
26.68 53.66202734457192 2 4
56.25826443045831 74.25826443045831 4 5
89.77887859998427 107.77887859998427 5 8
118.85623935722083 136.85623935722083 8 11
149.78666407941546 167.78666407941546 11 13
171.15465075593542 189.15465075593542 13 17
207.38 225.38 17 20
233.54285533564368 251.54285533564368 20 22
264.97659669765335 289.6844823508912 22 25
296.0084000832983 314.0084000832983 25 26
316.42 334.42 26 29
346.56 364.56 29 33
377.01276399668944 395.01276399668944 33 37
408.16968336601843 426.16968336601843 37 38
431.3486262510255 449.3486262510255 38 40
455.45194686725

16.36 34.36 0 3
36.441141548367966 63.94188187831318 3 5
90.20614981520575 149.0415414675311 5 7
151.23241232194587 169.23241232194587 7 16
189.7836220793035 219.08043969618132 16 19
223.02132779521773 241.02132779521773 19 21
252.09412015770022 291.4809408667228 21 23
292.1784260153943 310.1784260153943 23 29
316.6550989428581 349.70631003579894 29 31
353.1170304839306 371.1170304839306 31 35
374.88823610326386 402.2117337134176 35 36
408.57601988259495 458.2827954821362 36 40
470.48766714972464 496.32483377572254 40 41
500.18152537437965 511.22185459817723 41 47
ivn010-2023-02-27-Pula_06_1
0.0 18.0 0 3
38.74161159110656 56.74161159110656 3 4
105.04704351808465 123.04704351808465 4 6
133.42934769847628 151.42934769847628 6 8
161.45525952716062 179.45525952716062 8 9
184.5371010244322 202.5371010244322 9 12
224.18861307448196 242.18861307448196 12 15
245.50879054067747 270.62969970872825 15 17
338.22186074806064 356.22186074806064 17 18
394.14612553343375 412.14612553343375 18 20
488.5

193.01284215543095 218.91537051177977 19 23
219.08905073492795 256.5014551760724 23 25
257.79993731823214 280.04659794277194 25 26
281.33121587038517 305.927890344137 26 28
307.6087226042541 360.419898106176 28 32
363.43751655266055 393.61154850555164 32 33
393.61154850555164 413.8642176691063 33 39
413.8642176691063 435.96945690615405 39 41
435.96945690615405 453.96945690615405 41 47
461.3560491906126 512.6728036664587 47 49
512.6728036664587 533.0906805134277 49 53
543.8724802344085 567.0317878146386 53 55
ivn009-2023-02-15-Pula_06_1
32.52638078118798 50.52638078118798 0 1
55.55913738044391 73.5591373804439 1 5
111.87699335586332 142.09881881775567 5 10
149.84672754094447 167.84672754094447 10 11
177.81335705484497 195.81335705484497 11 12
216.65773228768182 234.65773228768182 12 14
236.7 262.4519501133787 14 16
277.11366094429627 295.11366094429627 16 17
317.0610402790643 335.0610402790643 17 19
343.24 361.24 19 21
364.4 385.03222222222223 21 25
385.86745489696347 403.86745489696347

117.26867697833524 135.26867697833524 7 11
138.84468519144428 162.37008840915286 11 15
166.19996945881473 184.19996945881473 15 18
194.53886612090372 236.79707720707577 18 21
236.79707720707577 254.79707720707577 21 24
257.32681555255016 308.76 24 29
312.59721099725056 330.59721099725056 29 38
339.0507204575409 366.7 38 40
369.0940010761749 387.0940010761749 40 50
389.1763692861581 416.7691069558363 50 54
417.38807763052915 435.38807763052915 54 55
439.4047486357663 475.46856259405655 55 58
ivn008-2023-02-14-Rovinj_08_1
0.33743397856930507 18.337433978569305 0 2
31.96974543340025 49.96974543340025 2 3
58.76995137211115 76.76995137211115 3 6
211.27245026720075 229.27245026720075 6 7
284.9855282793661 302.9855282793661 7 8
377.3436000370228 395.3436000370228 8 9
402.13787271482204 420.13787271482204 9 10
469.79282804645794 487.79282804645794 10 11
501.8304981920691 520.0240595868582 11 14
ivn008-2023-02-14-Rovinj_08_2
6.317128634000005 28.926805991502725 0 1
29.982901202853927 56.4325042

In [26]:
df_chunk = pd.DataFrame()

df_chunk['chunk_name'] = chunk_name
df_chunk['file_tier'] = file_info
df_chunk['start'] = start
df_chunk['end'] = end
df_chunk['start_id'] = start_id
df_chunk['end_id'] = end_id
df_chunk

,chunk_name,file_tier,start,end,start_id,end_id
0,ivn011-2023-02-25-Rovinj_03_1_1,ivn011-2023-02-25-Rovinj_03_1,10.354614,28.354614,0,1
1,ivn011-2023-02-25-Rovinj_03_1_2,ivn011-2023-02-25-Rovinj_03_1,62.177870,80.177870,1,2
2,ivn011-2023-02-25-Rovinj_03_1_3,ivn011-2023-02-25-Rovinj_03_1,91.988884,109.988884,2,3
3,ivn011-2023-02-25-Rovinj_03_1_4,ivn011-2023-02-25-Rovinj_03_1,174.571169,192.571169,3,5
4,ivn011-2023-02-25-Rovinj_03_1_5,ivn011-2023-02-25-Rovinj_03_1,381.900772,399.900772,5,6
...,...,...,...,...,...,...
1381,ivn007-2023-02-09-Rovinj_05_2_16,ivn007-2023-02-09-Rovinj_05_2,449.739877,479.629090,45,48
1382,ivn007-2023-02-09-Rovinj_05_2_17,ivn007-2023-02-09-Rovinj_05_2,485.803942,515.468239,48,51
1383,ivn007-2023-02-09-Rovinj_05_2_18,ivn007-2023-02-09-Rovinj_05_2,522.210053,540.210053,51,54
1384,ivn007-2023-02-09-Rovinj_05_2_19,ivn007-2023-02-09-Rovinj_05_2,544.607440,572.418984,54,55


In [28]:
# get text for the chunks from original un-chunked data

def get_chunk_text(start_id, end_id, file_tier):
    df_curr = df_sent[df_sent['file_tier'] == file_tier] 
    text_list = df_curr['text'].tolist() # change to clean
    text_chunks = text_list[start_id : end_id]
    text = ' '.join(text_chunks)
    return text

get_chunk_text(0, 2, 'ckm002-2022-01-04-Crikvenica_05_1')

''

In [29]:
df_chunk['text'] = df_chunk.apply(lambda x: get_chunk_text(x.start_id, x.end_id, x.file_tier), axis = 1)
df_chunk

,chunk_name,file_tier,start,end,start_id,end_id,text
0,ivn011-2023-02-25-Rovinj_03_1_1,ivn011-2023-02-25-Rovinj_03_1,10.354614,28.354614,0,1,Ah. M-hm.
1,ivn011-2023-02-25-Rovinj_03_1_2,ivn011-2023-02-25-Rovinj_03_1,62.177870,80.177870,1,2,I ciama pojačanja per vinzer e non per parteci...
2,ivn011-2023-02-25-Rovinj_03_1_3,ivn011-2023-02-25-Rovinj_03_1,91.988884,109.988884,2,3,"In teoria, mi gavessi 'sto preconcetto che chi..."
3,ivn011-2023-02-25-Rovinj_03_1_4,ivn011-2023-02-25-Rovinj_03_1,174.571169,192.571169,3,5,"Co ti va per boteghe, banche e poste, cussì- N..."
4,ivn011-2023-02-25-Rovinj_03_1_5,ivn011-2023-02-25-Rovinj_03_1,381.900772,399.900772,5,6,"Sorvola una volta, sorvola due, la quarta niss..."
...,...,...,...,...,...,...,...
1381,ivn007-2023-02-09-Rovinj_05_2_16,ivn007-2023-02-09-Rovinj_05_2,449.739877,479.629090,45,48,"Sì, s- Rovigno FM. Sì. @ fassevo una trasmissi..."
1382,ivn007-2023-02-09-Rovinj_05_2_17,ivn007-2023-02-09-Rovinj_05_2,485.803942,515.468239,48,51,M-hm. Sì. @ le ultime volte so che iera abasta...
1383,ivn007-2023-02-09-Rovinj_05_2_18,ivn007-2023-02-09-Rovinj_05_2,522.210053,540.210053,51,54,"Grazie. ((laugh)) Grazie ((laugh)). Sì, sì, sì..."
1384,ivn007-2023-02-09-Rovinj_05_2_19,ivn007-2023-02-09-Rovinj_05_2,544.607440,572.418984,54,55,@ no voleria gufar ((breath)) però penso che- ...


In [30]:
df_sent

,index,file,text,xmin,xmax,interval,tier,sent,tier_sent,chunk_len,file_tier
0,0,ivn011-2023-02-25-Rovinj_03,Ah. M-hm.,10.354614,11.565379,2,1,1,1,1.210766,ivn011-2023-02-25-Rovinj_03_1
1,1,ivn011-2023-02-25-Rovinj_03,I ciama pojačanja per vinzer e non per parteci...,62.177870,68.087574,4,1,1,1,5.909704,ivn011-2023-02-25-Rovinj_03_1
2,2,ivn011-2023-02-25-Rovinj_03,"In teoria, mi gavessi 'sto preconcetto che chi...",91.988884,106.124379,6,1,1,1,14.135496,ivn011-2023-02-25-Rovinj_03_1
3,3,ivn011-2023-02-25-Rovinj_03,"Co ti va per boteghe, banche e poste, cussì- N...",174.571169,179.231026,8,1,1,1,4.659856,ivn011-2023-02-25-Rovinj_03_1
4,4,ivn011-2023-02-25-Rovinj_03,"Te vien de saludar, @ incominciar in- in diale...",185.161761,190.070606,10,1,1,1,4.908845,ivn011-2023-02-25-Rovinj_03_1
...,...,...,...,...,...,...,...,...,...,...,...
3566,3566,ivn007-2023-02-09-Rovinj_05,Grazie ((laugh)).,528.478089,529.378438,105,2,1,1,0.900349,ivn007-2023-02-09-Rovinj_05_2
3567,3567,ivn007-2023-02-09-Rovinj_05,"Sì, sì, sì. Me x-",535.568677,537.417671,107,2,1,1,1.848995,ivn007-2023-02-09-Rovinj_05_2
3568,3568,ivn007-2023-02-09-Rovinj_05,@ no voleria gufar ((breath)) però penso che- ...,544.607440,572.418984,109,2,1,1,27.811544,ivn007-2023-02-09-Rovinj_05_2
3569,3569,ivn007-2023-02-09-Rovinj_05,"N- no, no, non- non me vedo el tipo de lavorar...",574.314667,579.389833,111,2,1,1,5.075166,ivn007-2023-02-09-Rovinj_05_2


In [31]:
df_chunk.to_csv(save_path + '/ivn_chunk_text_info_1.csv')

# get audio chunks

In [142]:
# get audio files according tp df_chunk info

save_chunk_path = save_path + '/cr_audio_chunk/'

In [144]:
# read in the audio, for each row in df_chunk, get the chunked audio and save
from pydub import AudioSegment

def get_audio_chunk(start, end, chunk_name, file_tier):
    audio_dir = folder_path +'/'+ file_tier[:-2] + '.wav'
    t1 = start * 1000 # s to ms
    t2 = end * 1000
    newAudio = AudioSegment.from_wav(audio_dir)[t1:t2]
    newAudio.export(save_chunk_path + chunk_name + '.wav', format="wav")
    return chunk_name + '.wav'

get_audio_chunk(0, 10, 'ckm002-2022-01-04-Crikvenica_05_1_1', 'ckm002-2022-01-04-Crikvenica_05_1')    

'ckm002-2022-01-04-Crikvenica_05_1_1.wav'

In [145]:
df_chunk['audio_name'] = df_chunk.apply(lambda x: get_audio_chunk(x.start, x.end, x.chunk_name, x.file_tier), axis = 1)
df_chunk

,chunk_name,file_tier,start,end,start_id,end_id,text,audio_name
0,ckm002-2022-01-04-Crikvenica_05_1_1,ckm002-2022-01-04-Crikvenica_05_1,31.204190,51.204190,0,1,Ste voleli školu?,ckm002-2022-01-04-Crikvenica_05_1_1.wav
1,ckm002-2022-01-04-Crikvenica_05_1_2,ckm002-2022-01-04-Crikvenica_05_1,65.324119,79.870934,1,3,Dal' van je baš neka njegova anegdota ili neč ...,ckm002-2022-01-04-Crikvenica_05_1_2.wav
2,ckm002-2022-01-04-Crikvenica_05_1_3,ckm002-2022-01-04-Crikvenica_05_1,172.354525,191.136708,3,5,"A kad ste malo divojke porasle, kako je to neg...",ckm002-2022-01-04-Crikvenica_05_1_3.wav
3,ckm002-2022-01-04-Crikvenica_05_1_4,ckm002-2022-01-04-Crikvenica_05_1,218.237587,237.273134,5,7,"A tu se trebalo i dobro snać, jer tad ni bilo ...",ckm002-2022-01-04-Crikvenica_05_1_4.wav
4,ckm002-2022-01-04-Crikvenica_05_1_5,ckm002-2022-01-04-Crikvenica_05_1,282.554819,302.554819,7,8,Ste voleli tancat?,ckm002-2022-01-04-Crikvenica_05_1_5.wav
...,...,...,...,...,...,...,...,...
876,ckm005-2022-11-26-Vrbnik_06_2_22,ckm005-2022-11-26-Vrbnik_06_2,481.870624,504.014365,117,124,"((hhh)) svi za njin. Onda bi bil, blo po tancu...",ckm005-2022-11-26-Vrbnik_06_2_22.wav
877,ckm005-2022-11-26-Vrbnik_06_2_23,ckm005-2022-11-26-Vrbnik_06_2,504.014365,524.812835,124,130,A izmedžu se @ kanta @ da se zabavi @ malo sop...,ckm005-2022-11-26-Vrbnik_06_2_23.wav
878,ckm005-2022-11-26-Vrbnik_06_2_24,ckm005-2022-11-26-Vrbnik_06_2,525.281052,537.900897,130,134,malo @ pokazat na placi da judi vide to bi bil...,ckm005-2022-11-26-Vrbnik_06_2_24.wav
879,ckm005-2022-11-26-Vrbnik_06_2_25,ckm005-2022-11-26-Vrbnik_06_2,538.139161,558.497226,134,137,"Ni @ valjda mladi il ki ne vole sopeli, ne vol...",ckm005-2022-11-26-Vrbnik_06_2_25.wav


In [ ]:
# goal: data info dataframe, cols are audio address, audio text, audio original time info and file info

